## Load Data

In [2]:
## Initiate Analysis by loading dataset and spliting

import numpy as np
import pandas as pd
import seaborn as sns
sns.set()
import matplotlib
from matplotlib import pyplot as plt
from sklearn import preprocessing

import warnings
warnings.simplefilter('ignore')

In [13]:
data = pd.read_csv("data.csv")

In [14]:
#split into test and train
test = data[pd.isnull(data['shot_made_flag'])] 
data2 = data.copy()
data.dropna(axis=0, how='any',inplace=True)

## Prepare Dataset

In [15]:
data.drop('team_id', axis=1, inplace=True) # same number
data.drop('shot_id', axis=1, inplace=True) # same number
data.drop('game_id', axis=1, inplace=True) # just some number with no effect to ds, like id
data.drop('game_event_id', axis=1, inplace=True) # sth like id
data.drop('team_name', axis=1, inplace=True) # Always LA Lakers


In [16]:
data['time_remaining'] = data['minutes_remaining'] * 60 + data['seconds_remaining']
#combine minutes and seconds into time in seconds

In [17]:
data.drop(['minutes_remaining', 'seconds_remaining', 'matchup'], axis=1, inplace=True) # converted


## Exploratory Analysis

In [18]:
data.drop(['lat', 'lon', 'game_date','loc_x','loc_y'], axis=1, inplace=True)

In [19]:
cumsum = data.groupby('season')['shot_made_flag'].cumsum() - data['shot_made_flag']
cumcnt = data.groupby('season').cumcount()
data['season'] = cumsum/cumcnt
data['season'].fillna(0, inplace = True)

In [20]:
cumsum = data.groupby('shot_zone_area')['shot_made_flag'].cumsum() - data['shot_made_flag']
cumcnt = data.groupby('shot_zone_area').cumcount()
data['shot_zone_area'] = cumsum/cumcnt
data['shot_zone_area'].fillna(0, inplace = True)

In [21]:
cumsum = data.groupby('shot_zone_range')['shot_made_flag'].cumsum() - data['shot_made_flag']
cumcnt = data.groupby('shot_zone_range').cumcount()
data['shot_zone_range'] = cumsum/cumcnt
data['shot_zone_range'].fillna(0, inplace = True)

In [22]:
cumsum = data.groupby('combined_shot_type')['shot_made_flag'].cumsum() - data['shot_made_flag']
cumcnt = data.groupby('combined_shot_type').cumcount()
data['combined_shot_type'] = cumsum/cumcnt
data['combined_shot_type'].fillna(0, inplace = True)

In [23]:
cumsum = data.groupby('shot_type')['shot_made_flag'].cumsum() - data['shot_made_flag']
cumcnt = data.groupby('shot_type').cumcount()
data['shot_type'] = cumsum/cumcnt
data['shot_type'].fillna(0, inplace = True)

In [24]:
cumsum = data.groupby('shot_zone_basic')['shot_made_flag'].cumsum() - data['shot_made_flag']
cumcnt = data.groupby('shot_zone_basic').cumcount()
data['shot_zone_basic'] = cumsum/cumcnt
data['shot_zone_basic'].fillna(0, inplace = True)

In [25]:
cumsum = data.groupby('action_type')['shot_made_flag'].cumsum() - data['shot_made_flag']
cumcnt = data.groupby('action_type').cumcount()
data['action_type'] = cumsum/cumcnt
data['action_type'].fillna(0, inplace = True)

In [26]:
cumsum = data.groupby('opponent')['shot_made_flag'].cumsum() - data['shot_made_flag']
cumcnt = data.groupby('opponent').cumcount()
data['opponent'] = cumsum/cumcnt
data['opponent'].fillna(0, inplace = True)

In [27]:
cumsum = data.groupby('time_remaining')['shot_made_flag'].cumsum() - data['shot_made_flag']
cumcnt = data.groupby('time_remaining').cumcount()
data['time_remaining'] = cumsum/cumcnt
data['time_remaining'].fillna(0, inplace = True)

In [28]:
cumsum = data.groupby('period')['shot_made_flag'].cumsum() - data['shot_made_flag']
cumcnt = data.groupby('period').cumcount()
data['period'] = cumsum/cumcnt
data['period'].fillna(0, inplace = True)

In [29]:
cumsum = data.groupby('playoffs')['shot_made_flag'].cumsum() - data['shot_made_flag']
cumcnt = data.groupby('playoffs').cumcount()
data['playoffs'] = cumsum/cumcnt
data['playoffs'].fillna(0, inplace = True)

In [30]:
from sklearn.model_selection import cross_val_score, KFold
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV

In [34]:
from sklearn.linear_model import LogisticRegressionCV
LogisticRegressionCV(Cs=10)
from sklearn.model_selection import train_test_split
train_X,test_X, train_Y, test_Y = train_test_split(data.drop('shot_made_flag',axis=1), 
                                                    data['shot_made_flag'], test_size=0.30, 
                                                    random_state=101)
from sklearn import metrics
model = LogisticRegressionCV()
model.fit(train_X,train_Y)
prediction=model.predict(test_X)
print('The accuracy of the Logistic Regression is',metrics.roc_auc_score(prediction,test_Y))

The accuracy of the Logistic Regression is 0.6911564823903242


In [32]:
X = data.drop('shot_made_flag',axis=1)
y = data.shot_made_flag

In [35]:
kfold = KFold(n_splits=5) # 5-fold CV
cv = kfold.split(X, y) # show data & make the split

clf = LogisticRegression()
scores = cross_val_score(clf, X, y, scoring='roc_auc', cv=cv)
print('KFold score: {0:.3f} ± {1:.3f}'.format(scores.mean(), scores.std()) )

KFold score: 0.677 ± 0.009
